In [2]:
import geopandas as gpd
import pandas as pd
from research.connections.db_client import DBClient
pd.set_option('display.max_columns', None)

In [3]:
db_client = DBClient()

In [4]:
sql_query = "SELECT * FROM public.wdpa_wdoecm_subset"

In [5]:
wdpa_subset = db_client.read_sql(sql_query, geom_col='shape')

In [35]:
wdpa_subset.head()

,id,wdpaid,wdpa_pid,pa_def,name,orig_name,desig,desig_eng,desig_type,iucn_cat,int_crit,marine,rep_m_area,gis_m_area,rep_area,gis_area,no_take,no_tk_area,status,status_yr,gov_type,own_type,mang_auth,mang_plan,cons_obj,supp_info,verif,metadataid,sub_loc,parent_iso3,iso3,shape_length,shape_area,shape,release_public,release_licenced,release_restricted,geom,radius_rep_area,radius
0,6640,555513089.0,555513089,1,Trutg,Trutg,Bundesinventar der Trockenwiesen und -weiden v...,Federal Inventory of Dry Grasslands and Pastur...,National,IV,Not Applicable,0,0.0,0.0,0.003461,0.084192,Not Applicable,0.0,Designated,2010,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,CHE,None,0.000010,"POLYGON ((9.15934 46.71844, 9.15939 46.71852, ...",None,None,None,0103000020E610000001000000A2010000E0E4D5859451...,1.213462e+06,33.19
1,6642,555513197.0,555513197,1,Rüs,Rüs,Bundesinventar der Trockenwiesen und -weiden v...,Federal Inventory of Dry Grasslands and Pastur...,National,IV,Not Applicable,0,0.0,0.0,0.001574,0.031669,Not Applicable,0.0,Designated,2010,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,CHE,None,0.000004,"POLYGON ((9.38225 46.72686, 9.38225 46.72687, ...",None,None,None,0103000020E6100000020000003701000040910AF1B5C3...,2.511153e+05,22.39
2,6646,555513215.0,555513215,1,Runchols,Runchols,Bundesinventar der Trockenwiesen und -weiden v...,Federal Inventory of Dry Grasslands and Pastur...,National,IV,Not Applicable,0,0.0,0.0,0.002909,0.039946,Not Applicable,0.0,Designated,2010,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,CHE,None,0.000005,"POLYGON ((9.68734 46.67611, 9.68736 46.67614, ...",None,None,None,0103000020E610000001000000AC010000807F8662EB5F...,8.576401e+05,30.43
3,6654,555513123.0,555513123,1,Monti di San Vittore,Monti di San Vittore,Bundesinventar der Trockenwiesen und -weiden v...,Federal Inventory of Dry Grasslands and Pastur...,National,IV,Not Applicable,0,0.0,0.0,0.002198,0.049098,Not Applicable,0.0,Designated,2010,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,CHE,None,0.000006,"MULTIPOLYGON (((9.11461 46.25393, 9.11457 46.2...",None,None,None,0106000020E610000002000000010300000003000000D0...,4.896792e+05,26.45
4,6677,555513315.0,555513315,1,Rungs,Rungs,Bundesinventar der Trockenwiesen und -weiden v...,Federal Inventory of Dry Grasslands and Pastur...,National,IV,Not Applicable,0,0.0,0.0,0.001243,0.019475,Not Applicable,0.0,Designated,2010,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,CHE,None,0.000002,"MULTIPOLYGON (((9.56033 46.64536, 9.56039 46.6...",None,None,None,0106000020E610000002000000010300000001000000C0...,1.565707e+05,19.89


# **1. AREA STATISTICS** 
- Exclude proposed areas

In [65]:
wdpa_subset.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7591 entries, 0 to 7590
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   id                  7591 non-null   int64   
 1   wdpaid              7591 non-null   float64 
 2   wdpa_pid            7591 non-null   object  
 3   pa_def              7591 non-null   object  
 4   name                7591 non-null   object  
 5   orig_name           7591 non-null   object  
 6   desig               7591 non-null   object  
 7   desig_eng           7591 non-null   object  
 8   desig_type          7591 non-null   object  
 9   iucn_cat            7591 non-null   object  
 10  int_crit            7591 non-null   object  
 11  marine              7591 non-null   object  
 12  rep_m_area          7591 non-null   float64 
 13  gis_m_area          7578 non-null   float64 
 14  rep_area            7591 non-null   float64 
 15  gis_area            7578 non-n

In [6]:
wdpa_subset.status.value_counts()

status
Designated    7579
Adopted          8
Inscribed        4
Name: count, dtype: int64

There are no WPDA `proposed` areas, so every value should be taking in account


The dataset has points and polygons. We should check this and transform the points into polygons.

In [7]:
query= """select distinct st_geometrytype(shape)
from public.wdpa_wdoecm_subset;"""

In [8]:
geom_types = db_client.read_sql(query)

In [9]:
geom_types

,st_geometrytype
0,ST_MultiPoint
1,ST_Polygon
2,ST_MultiPolygon


## Transform the data

As the ```WDPA_WDOECM_Manual```indicates on **5.5.2 Known issues Point Data** says 

>"If the area of a point feature has not been reported, it may be best to exclude it. To do this, users should remove points where the
```REP_AREA``` is zero. The remaining points can be buffered by calculating the radius of a circle
proportional to the reported area of the site using GIS tools.


In [32]:
rep_area_0 = """ SELECT t.name,t.rep_area,iso3,st_geometrytype(t.shape) FROM public.wdpa_wdoecm_subset as t
WHERE rep_area <= 0.01
ORDER BY id ASC"""

In [33]:
excluded_ones = db_client.read_sql(rep_area_0)
excluded_ones

,name,rep_area,iso3,st_geometrytype
0,Vedat de caça d'Enclar,0.000000,AND,ST_Polygon
1,Vedat de caça de Ransol,0.000000,AND,ST_Polygon
2,Vedat de caça de Xixerella,0.000000,AND,ST_Polygon
3,Vedat de caça del Parc Natural de la Vall de S...,0.000000,AND,ST_Polygon
4,Archipel Des Embiez - Six Fours,0.000000,FRA,ST_Polygon
...,...,...,...,...
1283,Hohstetten,0.002793,CHE,ST_Polygon
1284,Pfynwald,0.000373,CHE,ST_MultiPolygon
1285,Brigerbad,0.008962,CHE,ST_Polygon
1286,Jaggaseiu,0.008880,CHE,ST_MultiPolygon


In [38]:
table_ok = """
CREATE TABLE wdpa AS
SELECT *
FROM public.wdpa_wdoecm_subset
WHERE rep_area <> 0;
"""

In [39]:
db_client.run_in_transaction(table_ok)

In [12]:
sql_query = "SELECT * FROM public.wdpa"

In [34]:
wdpa = db_client.read_sql(sql_query, geom_col='geom')

In [37]:
wdpa.head.plot()

AttributeError: 'function' object has no attribute 'plot'

In [ ]:
db_client.run_in_transaction(query_buffer)

### Calculate BUFFER points
Radius proportional to the rep_area

```sqrt(rep_area / 3.1416)* 1000```

In [28]:
sql_query = "SELECT * FROM public.wdpa"
wdpa = db_client.read_sql(sql_query, geom_col='geom')
wdpa.head()

,id,wdpaid,wdpa_pid,pa_def,name,orig_name,desig,desig_eng,desig_type,iucn_cat,int_crit,marine,rep_m_area,gis_m_area,rep_area,gis_area,no_take,no_tk_area,status,status_yr,gov_type,own_type,mang_auth,mang_plan,cons_obj,supp_info,verif,metadataid,sub_loc,parent_iso3,iso3,shape_length,shape_area,shape,release_public,release_licenced,release_restricted,radius_area,geom
0,3620,32694.0,32694,1,Parco naturale del Mont Avic,Parco naturale del Mont Avic,Parco naturale regionale/provinciale,Regional/Provincial Nature Park,National,IV,Not Applicable,0,0.0,0.0,57.470000,57.687100,Not Applicable,0.0,Designated,1989,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,ITA,None,0.006658,0103000020E610000001000000D603000080C878944A68...,None,None,None,135.25,"POLYGON ((7.60185 45.70027, 7.60267 45.69808, ..."
1,4582,555512607.0,555512607,1,Mettje,Mettje,Bundesinventar der Trockenwiesen und -weiden v...,Federal Inventory of Dry Grasslands and Pastur...,National,IV,Not Applicable,0,0.0,0.0,0.102300,0.102271,Not Applicable,0.0,Designated,2010,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,CHE,None,0.000012,0103000020E6100000010000005506000000E46688C8D6...,None,None,None,5.71,"POLYGON ((7.70975 46.31738, 7.70978 46.31738, ..."
2,4935,149382.0,149382,1,Pian Segna,Pian Segna,Bundesinventar der Flachmoore von nationaler B...,Federal Inventory of Fenlands of National Impo...,National,IV,Not Applicable,0,0.0,0.0,0.011161,0.011155,Not Applicable,0.0,Designated,1994,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,CHE,None,0.000001,0106000020E61000000200000001030000000100000015...,None,None,None,1.88,"MULTIPOLYGON (((8.63851 46.18149, 8.63853 46.1..."
3,3080,555739292.0,555739292,1,SOUS LES MONTEES - Parcelle acquise en maitris...,SOUS LES MONTEES - Parcelle acquise en maitris...,Terrain acquis par un conservatoire d'espaces ...,Land acquired by a regional conservatory of na...,National,IV,Not Applicable,0,0.0,0.0,0.011412,0.011434,Not Applicable,0.0,Designated,0,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,FRA,None,0.000001,0103000020E6100000010000000800000040794576F96A...,None,None,None,1.91,"POLYGON ((5.85447 46.47006, 5.85314 46.47042, ..."
4,4723,31058.0,31058,1,Le Noirmont,Le Noirmont,Bundesinventar der eidgenössischen Jagdbanngeb...,Federal Hunting Reserves,National,IV,Not Applicable,0,0.0,0.0,37.613853,37.608759,Not Applicable,0.0,Designated,1992,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Applicable,Not Applicable,State Verified,2013,Not Reported,None,CHE,None,0.004408,0103000020E6100000010000003007000000C074D936B6...,None,None,None,109.42,"POLYGON ((6.17794 46.54588, 6.17819 46.54587, ..."


## Load Base Layer

In [14]:
base_layer = gpd.read_file('../data/base_layer_subset/base_layer_subset.shp')

In [8]:
base_layer.to_postgis('base_layer',con=db_client.engine)

In [16]:
query_base = "SELECT * FROM public.base_layer"
base_layer = db_client.read_sql(query_base, geom_col='geometry')

In [17]:
base_layer.head()

,OBJECTID,ISO3CD,ROMNAM,ADM1NM,POL_TYPE,TYPE,Shape_Leng,Shape_Area,MAP_LABEL,MAP_COLOR,STATUS,id,geometry
0,5.0,LIE,Liechtenstein,Liechtenstein,Land,Land,0.781427,0.018972,Liechtenstein,LIE,Member State,1,"POLYGON ((9.55550 47.22291, 9.55847 47.22388, ..."
1,36.0,GGY,Guernsey,None,200NM,EEZ,5.974343,0.812747,Guernsey (UK),GBR,Territory,1,"POLYGON ((-2.08791 49.42250, -2.08797 49.42243..."
2,43.0,GBR,United Kingdom of Great Britain & Northern Ire...,None,200NM,EEZ,350.648409,107.718373,United Kingdom of Great Britain & Northern Ire...,GBR,Member State,1,"POLYGON ((-0.48182 50.13153, -0.50136 50.12381..."
3,44.0,HRV,Croatia,None,200NM,EEZ,72.472037,6.165992,Croatia,HRV,Member State,1,"MULTIPOLYGON (((14.08333 43.95833, 14.06174 43..."
4,47.0,JEY,Jersey,None,200NM,EEZ,3.210346,0.280519,Jersey (UK),GBR,Territory,1,"POLYGON ((-2.55723 49.21585, -2.54154 49.22815..."
